- imports
- load dataset

In [36]:
import os
import pandas as pd

In [37]:
spotify_filepath = os.path.join(os.getcwd(),'data','Spotify_Dataset_V3.csv')
print(spotify_filepath)

spotify_data = pd.read_csv(spotify_filepath, delimiter=";")
spotify_data.head()

/Users/neo/Documents/1-app-ml/aml-code/AML-spotify/data/Spotify_Dataset_V3.csv


,Rank,Title,Artists,Date,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Valence,# of Artist,Artist (Ind.),# of Nationality,Nationality,Continent,Points (Total),Points (Ind for each Artist/Nat),id,Song URL
0,1,Ella Baila Sola,"Eslabon Armado, Peso Pluma",29/05/2023,0.668,0.758,-5176.0,0.033,0.483,0.000,0.834,Artist 1,Eslabon Armado,Nationality 1,Mexico,Latin-America,200,100.0,3qQbCzHBycnDpGskqOWY0E,https://open.spotify.com/track/3qQbCzHBycnDpGs...
1,1,Ella Baila Sola,"Eslabon Armado, Peso Pluma",29/05/2023,0.668,0.758,-5176.0,0.033,0.483,0.000,0.834,Artist 2,Peso Pluma,Nationality 2,Mexico,Latin-America,200,100.0,3qQbCzHBycnDpGskqOWY0E,https://open.spotify.com/track/3qQbCzHBycnDpGs...
2,2,WHERE SHE GOES,Bad Bunny,29/05/2023,0.652,0.800,-4019.0,0.061,0.143,0.629,0.234,Artist 1,Bad Bunny,Nationality 1,Puerto Rico,Latin-America,199,199.0,7ro0hRteUMfnOioTFI5TG1,https://open.spotify.com/track/7ro0hRteUMfnOio...
3,3,La Bebe - Remix,"Yng Lvcas, Peso Pluma",29/05/2023,0.812,0.479,-5678.0,0.333,0.213,0.000,0.559,Artist 1,Yng Lvcas,Nationality 1,Mexico,Latin-America,198,99.0,2UW7JaomAMuX9pZrjVpHAU,https://open.spotify.com/track/2UW7JaomAMuX9pZ...
4,3,La Bebe - Remix,"Yng Lvcas, Peso Pluma",29/05/2023,0.812,0.479,-5678.0,0.333,0.213,0.000,0.559,Artist 2,Peso Pluma,Nationality 2,Mexico,Latin-America,198,99.0,2UW7JaomAMuX9pZrjVpHAU,https://open.spotify.com/track/2UW7JaomAMuX9pZ...


data cleaning:
- all values are non-null, no need to remove rows
- should change "Date" column type to datetime
- dates seem reversed

In [38]:
spotify_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651936 entries, 0 to 651935
Data columns (total 20 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Rank                              651936 non-null  int64  
 1   Title                             651936 non-null  object 
 2   Artists                           651936 non-null  object 
 3   Date                              651936 non-null  object 
 4   Danceability                      651936 non-null  float64
 5   Energy                            651936 non-null  float64
 6   Loudness                          651936 non-null  float64
 7   Speechiness                       651936 non-null  float64
 8   Acousticness                      651936 non-null  float64
 9   Instrumentalness                  651936 non-null  float64
 10  Valence                           651936 non-null  float64
 11  # of Artist                       651936 non-null  o

In [42]:
# make "Date" column datetime type
spotify_data["Date"] = pd.to_datetime(spotify_data["Date"])
spotify_data.info()
spotify_data.loc[0]["Date"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651936 entries, 0 to 651935
Data columns (total 20 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   Rank                              651936 non-null  int64         
 1   Title                             651936 non-null  object        
 2   Artists                           651936 non-null  object        
 3   Date                              651936 non-null  datetime64[ns]
 4   Danceability                      651936 non-null  float64       
 5   Energy                            651936 non-null  float64       
 6   Loudness                          651936 non-null  float64       
 7   Speechiness                       651936 non-null  float64       
 8   Acousticness                      651936 non-null  float64       
 9   Instrumentalness                  651936 non-null  float64       
 10  Valence                         

Timestamp('2023-05-29 00:00:00')

In [ ]:
# TODO: reverse dates

random thoughts:
- rank decides the points and the points for the song are shared equally by all the artists on the track.
- why are all the song aspects [0, 1] but Loudness is [-34475, 1509]
- Find top 5 artists in terms of points every year
- Find top 5 artists for every region?

In [43]:
spotify_data.describe()

,Rank,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Valence,Points (Total),Points (Ind for each Artist/Nat)
count,651936.000000,651936.000000,651936.000000,651936.000000,651936.000000,651936.000000,651936.000000,651936.000000,651936.000000,651936.000000
mean,100.407782,0.697730,0.652049,-5295.645217,0.109946,0.230694,0.007647,0.523134,100.592218,72.391141
std,57.406664,0.133186,0.155684,2785.261321,0.096841,0.230710,0.054724,0.223926,57.406664,54.227892
min,1.000000,0.073000,0.005000,-34475.000000,0.022000,0.000000,0.000000,0.026000,1.000000,0.200000
25%,51.000000,0.617000,0.549000,-6825.000000,0.045000,0.048000,0.000000,0.356000,51.000000,28.000000
50%,100.000000,0.719000,0.671000,-5206.000000,0.068000,0.152000,0.000000,0.521000,101.000000,60.000000
75%,150.000000,0.793000,0.771000,-3885.000000,0.136000,0.349000,0.000000,0.696000,150.000000,104.000000
max,200.000000,0.985000,0.996000,1509.000000,0.966000,0.994000,0.956000,0.982000,200.000000,200.000000


In [50]:
# checking random artists

# print(spotify_data[spotify_data["Artists"] == "Ed Sheeran"].head())
spotify_data[spotify_data["Artists"] == "Taylor Swift"]

,Rank,Title,Artists,Date,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Valence,# of Artist,Artist (Ind.),# of Nationality,Nationality,Continent,Points (Total),Points (Ind for each Artist/Nat),id,Song URL
17,13,Anti-Hero,Taylor Swift,2023-05-29,0.637,0.643,-6571.0,0.052,0.130,0.000,0.533,Artist 1,Taylor Swift,Nationality 1,United States,Anglo-America,188,188.0,0V3wPSX9ygBnCm8psDIegu,https://open.spotify.com/track/0V3wPSX9ygBnCm8...
48,30,Cruel Summer,Taylor Swift,2023-05-29,0.552,0.702,-5707.0,0.157,0.117,0.000,0.564,Artist 1,Taylor Swift,Nationality 1,United States,Anglo-America,171,171.0,1BxfuPKGuaTgP7aM0Bbdwr,https://open.spotify.com/track/1BxfuPKGuaTgP7a...
52,33,Hits Different,Taylor Swift,2023-05-29,0.672,0.782,-6499.0,0.042,0.148,0.000,0.239,Artist 1,Taylor Swift,Nationality 1,United States,Anglo-America,168,168.0,3xYJScVfxByb61dYHTwiby,https://open.spotify.com/track/3xYJScVfxByb61d...
77,51,Blank Space,Taylor Swift,2023-05-29,0.760,0.703,-5412.0,0.054,0.103,0.000,0.570,Artist 1,Taylor Swift,Nationality 1,United States,Anglo-America,150,150.0,1u8c2t2Cy7UBoG4ArRcF5g,https://open.spotify.com/track/1u8c2t2Cy7UBoG4...
89,59,cardigan,Taylor Swift,2023-05-29,0.613,0.581,-8588.0,0.042,0.537,0.000,0.551,Artist 1,Taylor Swift,Nationality 1,United States,Anglo-America,142,142.0,4R2kfaDFhslZEMJqAFNpdd,https://open.spotify.com/track/4R2kfaDFhslZEMJ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611060,165,Bad Blood,Taylor Swift,2017-09-06,0.652,0.802,-6114.0,0.181,0.087,0.000,0.295,Artist 1,Taylor Swift,Nationality 1,United States,Anglo-America,36,36.0,0TvQLMecTE8utzoNmvXRbK,https://open.spotify.com/track/0TvQLMecTE8utzo...
611067,122,Blank Space,Taylor Swift,2017-09-06,0.760,0.703,-5412.0,0.054,0.103,0.000,0.570,Artist 1,Taylor Swift,Nationality 1,United States,Anglo-America,79,79.0,1u8c2t2Cy7UBoG4ArRcF5g,https://open.spotify.com/track/1u8c2t2Cy7UBoG4...
611219,135,Shake It Off,Taylor Swift,2017-09-06,0.647,0.800,-5384.0,0.165,0.065,0.000,0.942,Artist 1,Taylor Swift,Nationality 1,United States,Anglo-America,66,66.0,0cqRj7pUJDkTCEsJkx8snD,https://open.spotify.com/track/0cqRj7pUJDkTCEs...
611248,162,Style,Taylor Swift,2017-09-06,0.588,0.791,-5595.0,0.040,0.002,0.003,0.487,Artist 1,Taylor Swift,Nationality 1,United States,Anglo-America,39,39.0,0ug5NqcwcFR2xrfTkc7k8e,https://open.spotify.com/track/0ug5NqcwcFR2xrf...
